<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



file1 = open(r"C:\tmp\BERT_results\zazu.txt", 'w')
L = ["This is Delhi\n", "This is Paris\n", "This is London\n"]
s = "Hello\n"
file1.write(s)
file1.writelines(L)
file1.close()
file1 = open(r"C:\tmp\BERT_results\zazu.txt", 'r')
print(file1.read())
file1.close()



!pwd
!cd /..
!pwd
!ls -la /content/ML_GitHub/BERT_results/checkpoint-80
!cat /content/ML_GitHub/BERT_results/checkpoint-80/trainer_state.json
from google.colab import files
files.download(r"/content/ML_GitHub/BERT_results/checkpoint-80/trainer_state.json")





In [1]:
from google.colab import userdata

import os

current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

dir_1000_125_125 = False
dir_128_18_54    = False

from google.colab import files
uploaded = files.upload()

!ls -la

Current directory: /content


Saving datasetHF_128_18_54.zip to datasetHF_128_18_54.zip
Saving skills.csv to skills.csv
total 236
drwxr-xr-x 1 root root   4096 Nov 14 17:33 .
drwxr-xr-x 1 root root   4096 Nov 14 17:32 ..
drwxr-xr-x 4 root root   4096 Nov 12 14:24 .config
-rw-r--r-- 1 root root 213003 Nov 14 17:33 datasetHF_128_18_54.zip
drwxr-xr-x 1 root root   4096 Nov 12 14:25 sample_data
-rw-r--r-- 1 root root   7805 Nov 14 17:33 skills.csv


In [2]:
import pandas as pd

# Load the CSV file
skill_df       = pd.read_csv("skills.csv")
skill_df['Id'] = skill_df['Id'].astype(str)

In [3]:
if os.path.isfile("datasetHF_1000_125_125.zip"):
    print("datasetHF_1000_125_125.zip exists")
    !unzip datasetHF_1000_125_125.zip -d datasetHF_1000_125_125
    dir_1000_125_125 = True
elif os.path.isfile("datasetHF_128_18_54.zip"):
    print("datasetHF_128_18_54.zip exists")
    !unzip datasetHF_128_18_54.zip -d datasetHF_128_18_54
    dir_128_18_54 = True
else:
  print("Neither datasetHF_1000_125_125.zip nor datasetHF_128_18_54.zip exists")

datasetHF_128_18_54.zip exists
Archive:  datasetHF_128_18_54.zip
   creating: datasetHF_128_18_54/test/
   creating: datasetHF_128_18_54/train/
   creating: datasetHF_128_18_54/validation/
  inflating: datasetHF_128_18_54/dataset_dict.json  
  inflating: datasetHF_128_18_54/test/data-00000-of-00001.arrow  
  inflating: datasetHF_128_18_54/test/dataset_info.json  
  inflating: datasetHF_128_18_54/test/state.json  
  inflating: datasetHF_128_18_54/train/data-00000-of-00001.arrow  
  inflating: datasetHF_128_18_54/train/dataset_info.json  
  inflating: datasetHF_128_18_54/train/state.json  
  inflating: datasetHF_128_18_54/validation/data-00000-of-00001.arrow  
  inflating: datasetHF_128_18_54/validation/dataset_info.json  
  inflating: datasetHF_128_18_54/validation/state.json  


In [4]:
#!pip install fsspec==2024.10.0
!pip install -q transformers datasets

from datasets import DatasetDict

if dir_1000_125_125:
    dataset = DatasetDict.load_from_disk('datasetHF_1000_125_125')
elif dir_128_18_54:
    dataset = DatasetDict.load_from_disk('datasetHF_128_18_54')
else:
    print("Neither dir datasetHF_1000_125_125 nor datasetHF_128_18_54 exists")

print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (128, 44), 'validation': (18, 44), 'test': (54, 44)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '394', '142', '146', '147', '148', '149', '150', '151', '408', '409', '153', '154', '155', '156', '157', '158', '160', '152', '162', '165', '

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's check the first example of the training split:

In [5]:
example = dataset['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

example: <class 'dict'> dict_keys(['id', 'text', '394', '142', '146', '147', '148', '149', '150', '151', '408', '409', '153', '154', '155', '156', '157', '158', '160', '152', '162', '165', '167', '168', '169', '170', '171', '685', '174', '686', '176', '689', '173', '356', '360', '361', '362', '364', '760', '756', '758', '375', '376', '761'])
{'id': 140409, 'text': "Inetum-Realdolmen - Azure Cloud Engineer Azure Cloud Inetum-Realdolmen YOUR FUNCTION To support the exponential growth of our Azure practice, we are looking for several Azure Cloud Engineers. Here's how you'll make impact: For larger projects you work in team with other Azure Cloud Engineers, Cloud Solution Architects and Project Managers to write a new success story. We can count on you for the professional implementation of the tasks entrusted to you. For smaller cloud projects you are in charge for the full engagement: you advise your client from the design until his solution is fully operational. In addition to consultan

The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [6]:
### if dataset 1000_125_125, 48 labels
### if dataset 128_18_54   , 42 labels

labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

filtered_skill_df = skill_df[skill_df['Id'].isin(labels)]
print(f"filtered_skill_df: {type(filtered_skill_df)} {filtered_skill_df.shape}\n{filtered_skill_df}")

id2label = {idx:label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label:idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

labels: <class 'list'> 42
['142', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '160', '162', '165', '167', '168', '169', '170', '171', '173', '174', '176', '356', '360', '361', '362', '364', '375', '376', '394', '408', '409', '685', '686', '689', '756', '758', '760', '761']
filtered_skill_df: <class 'pandas.core.frame.DataFrame'> (42, 3)
     Id  SkillTypeId                             Value
0   142            7    Developer / Analyst Programmer
2   146            7  Application / Solution Architect
3   147            7          Infrastructure Architect
4   148            7                 Technical Analyst
5   149            7                Functional Analyst
6   150            7        Test / Validation Engineer
7   151            7         Test / Validation Manager
8   152            7                  Technical Writer
9   153            7                Database Developer
10  154            7            Database Administrator
11  155  

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [7]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# examples and not example because batched=True => examples is a batch
def preprocess_data(examples, indices):
  # take a batch of texts
  text = examples["text"]
  #print("Indices:", indices, "Batch size:", len(text), "Num labels:", len(labels))

  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)

  labels_batch = {label: np.zeros(len(text)) for label in labels}                                                # validation: 42 rows: 'label': 18 x 0.0
  #print(f"labels_batch: zeros: {type(labels_batch)} {len(labels_batch)} {labels_batch}")
  #print(f"examples.keys(): {examples.keys()}")
  # examples.keys(): all the column names (keys) in the batch dict "examples"
  # for each k in labels, a new key-value pair k: examples[k] is added to labels_batch
  labels_batch.update({k: [1.0 if val else 0.0 for val in examples[k]] for k in examples.keys() if k in labels}) # validation: 42 rows: 'label': 1.0 or 0.0

  #print(f"labels_batch: updated: {type(labels_batch)} {len(labels_batch)} {labels_batch}")

  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))                                                             # validation: 18 rows 42 columns
  #print(f"labels_matrix:{type(labels_matrix)} {labels_matrix.shape}")
  # fill numpy array
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")                                                                            # labels are sorted
    labels_matrix[:, idx] = labels_batch[label] # for each row, sets the idx-th column of labels_matrix with the values from labels_batch[label]

  #print("First row of labels_matrix:", labels_matrix[0])

  # Add labels to encoding
  encoding["labels"] = labels_matrix.tolist()
  print(f"encoding['labels']: {encoding['labels']}")


  return encoding



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names, with_indices=True)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [11]:
example = encoded_dataset['validation'][0]
print(f"example.keys(): {example.keys()}")
print(f"example['input_ids']: {example['input_ids']}")
print(f"example['token_type_ids']: {example['token_type_ids']}")
print(f"example['attention_mask']: {example['attention_mask']}")
print(f"example['labels']: {example['labels']}")

example.keys(): dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
example['input_ids']: [101, 2552, 15735, 2015, 1011, 8040, 6824, 3040, 1013, 16216, 16643, 2239, 20589, 2139, 4013, 15759, 2015, 8040, 6824, 3040, 1010, 10026, 2139, 4013, 15759, 2015, 1010, 1012, 5658, 2552, 15735, 2015, 3393, 2326, 4503, 5051, 3672, 1040, 1005, 2552, 15735, 2015, 9765, 17202, 1040, 1005, 16655, 7990, 18175, 2139, 2711, 5267, 21864, 19817, 12462, 10484, 3372, 29032, 9515, 6216, 3672, 7505, 4078, 4013, 15759, 2015, 25272, 2595, 1006, 9998, 20236, 4310, 1010, 5127, 14192, 2063, 10364, 2474, 16216, 16643, 2239, 4078, 4262, 4372, 7913, 2552, 15735, 2015, 3802, 4649, 2112, 8189, 7442, 2015, 1010, 5127, 14192, 2063, 10364, 2474, 16216, 16643, 2239, 4078, 4262, 4372, 7913, 2552, 15735, 2015, 3802, 4649, 12666, 26744, 1010, 28616, 2063, 4372, 2173, 1040, 1005, 4895, 25272, 13675, 2213, 1010, 4385, 1012, 1007, 1012, 29536, 7913, 2535, 4372, 9092, 2102, 10861, 8040, 6824, 3040, 1013, 16216, 1

In [12]:
tokenizer.decode(example['input_ids'])

"[CLS] actiris - scrum master / gestionnaire de projets scrum master, chef de projets,. net actiris le service developpement d ' actiris est compose d ' une trentaine de personnes qui travaillent essentiellement sur des projets nouveaux ( dossier unique, plateforme pour la gestion des relations entre actiris et les partenaires, plateforme pour la gestion des relations entre actiris et les employeurs, mise en place d ' un nouveau crm, etc. ). votre role en tant que scrum master / gestionnaire de projets, vous initiez et coordonnez, dans un environnement agile, des projets dans le cadre du developpement de nos nouvelles plateformes core business et vous veillez a leur bonne realisation dans les delais impartis. vos responsabilites vous collectez les informations necessaires au bon deroulement du projet et etes capable de discuter d ' un backlog avec le metier ; vous determinez les etapes cles, planning et retro - planning highlevel sans forcement de requete aupres de l ' equipe de develo

In [13]:
example = encoded_dataset['validation'][0]
print(f"example['labels']: {example['labels']}")


example['labels']: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [14]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]


['169', '170', '375']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [15]:
encoded_dataset.set_format("torch")
example = encoded_dataset['validation'][0]
print(f"example['labels']:  {type(example['labels'])} {example['labels'].shape}\n{example['labels']}")

#raise Exception("STOP")

example['labels']:  <class 'torch.Tensor'> torch.Size([42])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])


## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
#raise Exception("STOP")

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [17]:
batch_size = 8
metric_name = "f1"

In [19]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir = r'C:\tmp\BERT_results\output',
    overwrite_output_dir=True,
    logging_dir= r'C:\tmp\BERT_results\logs',
    logging_steps=50,
    save_steps=100,
    save_total_limit=2,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
    run_name= "BERT-multilabel-lr1e5-epochs5-datasetHF_128_18_54"
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [20]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score, average_precision_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.2):
    _average = 'micro'    # 'micro' or 'weighted'
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true               = labels

    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=_average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=_average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)

    # return as dictionary
    metrics = {'f1'                  : f1,
               'precision'           : precision,
               'recall'              : recall,
               'roc_auc'             : roc_auc,
               'precision_recall_auc': precision_recall_auc,
               'accuracy'            : accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [21]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [22]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1999,  3388,  2819,  1011,  2613,  3527, 13728,  2368,  1011,
        24296,  6112,  3992, 24296,  6112,  1999,  3388,  2819,  1011,  2613,
         3527, 13728,  2368,  2115,  3853,  2000,  2490,  1996, 27258,  3930,
         1997,  2256, 24296,  3218,  1010,  2057,  2024,  2559,  2005,  2195,
        24296,  6112,  6145,  1012,  2182,  1005,  1055,  2129,  2017,  1005,
         2222,  2191,  4254,  1024,  2005,  3469,  3934,  2017,  2147,  1999,
         2136,  2007,  2060, 24296,  6112,  6145,  1010,  6112,  5576,  8160,
         1998,  2622, 10489,  2000,  4339,  1037,  2047,  3112,  2466,  1012,
         2057,  2064,  4175,  2006,  2017,  2005,  1996,  2658,  7375,  1997,
         1996,  8518, 18011,  2000,  2017,  1012,  2005,  3760,  6112,  3934,
         2017,  2024,  1999,  3715,  2005,  1996,  2440,  8147,  1024,  2017,
        18012,  2115,  7396,  2013,  1996,  2640,  2127,  2010,  5576,  2003,
         3929,  6515,  1012,  1999,  2804,  2000, 24853,  1010, 

In [23]:
#forward pass
print(f"inputids: {type(encoded_dataset['train']['input_ids'][0])} {encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask: {type(encoded_dataset['train']['attention_mask'][0])} {encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels: {type(encoded_dataset['train'][0]['labels'])} {encoded_dataset['train'][0]['labels'].shape}")

outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0),
                attention_mask=encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
                labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

inputids: <class 'torch.Tensor'> torch.Size([512])
attention_mask: <class 'torch.Tensor'> torch.Size([512])
labels: <class 'torch.Tensor'> torch.Size([42])


SequenceClassifierOutput(loss=tensor(0.7729, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-4.1564e-01, -3.9186e-01,  4.9481e-01,  5.8015e-01,  2.9665e-01,
          3.8690e-01,  1.9091e-02,  9.9899e-02,  4.8971e-01,  1.1852e-02,
         -2.6776e-02, -2.2577e-01, -6.0425e-02,  7.7346e-01, -5.4748e-01,
         -1.5658e-01, -6.7136e-01,  3.5473e-02,  9.3673e-01,  2.0104e-03,
         -2.8746e-02,  3.3417e-01,  1.4998e-01,  2.3222e-01, -3.7599e-01,
         -8.4940e-02, -2.4690e-02,  1.7599e-01, -3.1264e-01,  1.9375e-01,
          6.2560e-02,  1.8334e-01,  1.2381e-01,  2.7637e-01,  4.8331e-01,
          2.0044e-01,  7.4059e-04,  3.7642e-01,  9.9254e-02,  1.6765e-01,
          9.2317e-02,  4.2429e-01]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-24-c90a37d0d319>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,No log,0.622301,0.102886,0.054233,1.000000,0.533720,0.079039,0.000000
2,No log,0.559298,0.105263,0.055556,1.000000,0.519836,0.057227,0.000000
3,No log,0.510921,0.104592,0.055182,1.000000,0.549173,0.064332,0.000000
4,0.600000,0.487916,0.104725,0.055256,1.000000,0.552857,0.063647,0.000000
5,0.600000,0.480820,0.104994,0.055405,1.000000,0.554494,0.063661,0.000000


TrainOutput(global_step=80, training_loss=0.5611387968063355, metrics={'train_runtime': 147.1732, 'train_samples_per_second': 4.349, 'train_steps_per_second': 0.544, 'total_flos': 168451552051200.0, 'train_loss': 0.5611387968063355, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [26]:
eval_results = trainer.evaluate()

## Inference

Let's test the model on a new sentence:

id: 323697
"Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten,"

['142', '147', '149', '154', '156', '157', '173', '409', '685', '689']

---

id: 323611,"Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home","171,170,794,800,798,797,138,139,352"
---


In [25]:
#text = "Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten"
#text = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"
#encoding = tokenizer(text, return_tensors="pt")
#encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

#outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [26]:
#logits = outputs.logits
#logits.shape

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [27]:
# apply sigmoid + threshold
#import torch

#sigmoid = torch.nn.Sigmoid()
#probs = sigmoid(logits.squeeze().cpu())
#predictions = np.zeros(probs.shape)
#predictions[np.where(probs >= 0.2)] = 1
# turn predicted id's into actual label names
#predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
#print(predicted_labels)

#raise Exception("STOP")

**id**: 323697

**MySQL**: "142,189,190,754,208,794,676,811,812,139,138" (only 142="Developer / Analyst Programmer") is a 7-skill)

**predicted_labels**: ['148', '152', '154', '409'] : all are 7-skills: 148="Technical Analyst", 152="Technical Writer", 154="Database Admininistrator"


---


**id**: 323611

**MySQL**: "171,170,794,800,798,797,138,139,352"            
           171: Project Mgmt Officer (PMO)  
           170: Project Manager / Coordinator

**predicted labels**: 409:  
                      409: "SOA Specialist" (SOA: Service Oriented Architecture)

In [27]:
trainer.save_model("skills_model")    # Save locally the trained model and tokenizer: saves the model weights, the tokenizer, the model configuration file ("config.json")

import json

with open("training_metrics.json", 'w') as f:
    json.dump(trainer.state.log_history,f)

with open("eval_metrics.json", 'w') as f:
    json.dump(eval_results, f)


In [28]:
!pip install transformers huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [29]:
from huggingface_hub import create_repo, HfApi
from huggingface_hub.utils import RepositoryNotFoundError

repo_id = 'claudelepere/skills_model'
api     = HfApi()
try:
    api.repo_info(repo_id)
    print(f"repo_id: {repo_id}")
except RepositoryNotFoundError:
    create_repo(repo_id, private=True)
    print(f"Repo {repo_id} created succesfully as a private repo.")

repo_id: claudelepere/skills_model


In [30]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("skills_model")
tokenizer = AutoTokenizer.from_pretrained("skills_model")

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/claudelepere/skills_model/commit/bace606b7a0ef5795009fa7096295accee275293', commit_message='Upload tokenizer', commit_description='', oid='bace606b7a0ef5795009fa7096295accee275293', pr_url=None, repo_url=RepoUrl('https://huggingface.co/claudelepere/skills_model', endpoint='https://huggingface.co', repo_type='model', repo_id='claudelepere/skills_model'), pr_revision=None, pr_num=None)

In [ ]:
raise Exception("STOP")


In [31]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the trained model and tokenizer
model     = AutoModelForSequenceClassification.from_pretrained(repo_id)
tokenizer = AutoTokenizer.from_pretrained(repo_id)

config.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [37]:
#text      = "Sample text for prediction"
#text = "Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten"
text = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"

inputs    = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
threshold = 0.5

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.sigmoid(logits)
    #predictions = torch.where(probs >= threshold, torch.ones_like(probs), torch.zeros_like(probs))
    #predictions = torch.argmax(probs, dim=-1) if model.config.num_labels > 1 else torch.where(probs >= threshold, torch.ones_like(probs), torch.zeros_like(probs))
    preds = (probs > threshold).int()
    print(f"probs: {probs} preds: {preds}")
    print()
    for label, Value, prob, pred in zip(filtered_skill_df['Id'], filtered_skill_df['Value'], probs.squeeze(), preds.squeeze()):
      if (pred == 1):
        print(f"label: {label} prob: {prob.item():.4f} pred: {int(pred.item())} {Value}")


probs: tensor([[0.3197, 0.5208, 0.3680, 0.4562, 0.4166, 0.4250, 0.4821, 0.3839, 0.4007,
         0.3872, 0.3394, 0.4091, 0.4027, 0.3176, 0.3726, 0.4497, 0.4625, 0.4577,
         0.3557, 0.4252, 0.4913, 0.5360, 0.4487, 0.2910, 0.5642, 0.4465, 0.4409,
         0.4491, 0.4268, 0.4252, 0.4703, 0.4758, 0.3731, 0.4799, 0.4004, 0.3090,
         0.2463, 0.3407, 0.1796, 0.5750, 0.3712, 0.4141]]) preds: tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]],
       dtype=torch.int32)

label: 146 prob: 0.5208 pred: 1 Application / Solution Architect
label: 171 prob: 0.5360 pred: 1 Project Mgmt Officer (PMO)
label: 176 prob: 0.5642 pred: 1 Sales / Account Manager
label: 758 prob: 0.5750 pred: 1 Presales / Technical Sales


from google.colab import drive
drive.mount('/content/drive')

# Copy files to your Google Drive
!cp -r my_model /content/drive/MyDrive/
!cp training_metrics.json /content/drive/MyDrive/
!cp eval_metrics.json /content/drive/MyDrive/

In [ ]:
from google.colab import files

#!zip -r my_model.zip my_model
#!split -b 100M my_model.zip my_model_part_
#!unzip - my_model.zip

# for part in ['my_model_part_aa', 'my_model_part_ab', 'my_model_part_ac']:  # Adjust based on number of parts
#    files.download(part)
#files.download("my_model_part_aa")
#files.download("my_model_part_ab")
#files.download("my_model_part_ac")
#files.download("my_model_part_ad")
#!md5sum my_model.zip
#files.download("training_metrics.json")
#files.download("eval_metrics.json")

#uploaded = files.upload()
#!md5sum my_model.zip
#!md5sum training_metrics.json
#!md5sum eval_metrics.json

#!unzip my_model.zip -d my_model_unzip

#raise Exception("STOP")

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("/content/my_model_unzip/my_model")
tokenizer = AutoTokenizer.from_pretrained("/content/my_model_unzip/my_model")

import json
with open("/content/training_metrics.json", 'r') as f:
    training_metrics = json.load(f)
with open("/content/eval_metrics.json", 'r') as f:
    eval_metrics = json.load(f)



In [ ]:
from transformers import TrainingArguments, Trainer

#text = "Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten"
text = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"
encoding = tokenizer(text, return_tensors="pt")

# Define the device based on availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the device
model.to(device)
# Move encoding to the device of the model
encoding = {k: v.to(device) for k,v in encoding.items()}

# Perform inference
with torch.no_grad():    # no gradients needed for inference. Forward pass
    outputs = model(**encoding)

# Get logits from the model's output
logits = outputs.logits

# Apply softmax/sigmoid based on the type of classification
if model.config.num_labels == 1:
    probs = torch.sigmoid(logits.squeeze())
else:
    #probs = torch.softmax(logits, dim=1).squeeze()
    probs = torch.sigmoid(logits)



# To get predictions
threshold = 0.5
#predictions = torch.where(probs >= threshold, torch.ones_like(probs), torch.zeros_like(probs))
#predictions = torch.argmax(probs, dim=-1) if model.config.num_labels > 1 else torch.where(probs >= threshold, torch.ones_like(probs), torch.zeros_like(probs))
predictions = (probs > threshold).float()
print("Predictions:", predictions)
print()

# Turn predicted id's into actual label names
print("Probabilites:", probs)

#[id2label[idx] for idx, label in enumerate(predictions['labels']) if label == 1.0]

#predicted_labels = [id2label[idx.item()] for idx in predictions]
#print(predicted_labels)

for label, prob, pred in zip(labels, probs.squeeze(), predictions.squeeze()):
  print(f"Label: {label}: Probability: {prob.item():.4f} {int(pred.item())}")

